# Descarga de datos y preprocesado
En este código vamos a descargar los datos de las elecciones, rentas y datos demográficos para ser analizados posteriormente.

In [1]:
!cd .. && pip install .

^C


In [1]:
!conda install -c conda-forge -y folium

Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Programas\anaconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.10.0              |             py_1          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/noarch::folium-0.10.0-py_1
  vincent            conda-forge/noarch::vincent-0.4.4-py_1




vincent-0.4.4        | 28 KB     |            |   0% 
vincent-0.4.4        | 28 KB     | #####7     |  58% 
vinc

El sistema no puede encontrar la ruta especificada.


In [2]:
import os
import pandas as pd
import numpy as np
import bokeh as bk
import holoviews as hv
from tqdm import tqdm


from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column


current_dir = os.path.dirname(os.getcwd())
current_dir

'C:\\Users\\alexm\\Dropbox\\Otros\\espana_en_cifras'

## Descarga de datos

Primero descargamos todos los datos. Aquí se incluyen las elecciones de varios años, los datos de renta y datos demográficos.

In [11]:
from eec import sacar_datos

# Datos de las elecciones
# Estos datos tienen toda la información acerca de las elecciones. Las elecciones incluyen los partidos que se presentan,
# y el número de votos por colegio electoral y partido.
sacar_datos(current_dir + '/datos/congreso_2019_04/', '02201904_MESA.zip', 
            'http://www.infoelectoral.mir.es/infoelectoral/docxl/apliextr/02201904_MESA.zip')

sacar_datos(current_dir + '/datos/congreso_2016_06/', '02201606_MESA.zip', 
            'http://www.infoelectoral.mir.es/infoelectoral/docxl/apliextr/02201606_MESA.zip')

sacar_datos(current_dir + '/datos/congreso_2015_12/', '02201512_MESA.zip', 
            'http://www.infoelectoral.mir.es/infoelectoral/docxl/apliextr/02201512_MESA.zip')

# Datos censales 2016
# Estos datos tienen dos ficheros principales:
# 1) renta_media
# 2) indicadores_demográficos
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30824.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_c/30824.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30832.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_c/30832.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30831.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_c/30831.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30830.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_c/30830.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30828.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_c/30828.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30827.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_c/30827.csv")

# Datos censales 2011
# Este fichero contiene muchos datos demográficos; asociados a edad, sexo, inmigración, etc.
sacar_datos(current_dir + '/datos/censos2011_datos/', 'indicadores_seccion_censal_csv.zip',
           "https://www.ine.es/censos2011_datos/indicadores_seccion_censal_csv.zip")
sacar_datos(current_dir + '/datos/censos2011_datos/', 'indicadores_seccen_rejilla.xls',
           "https://www.ine.es/censos2011_datos/indicadores_seccen_rejilla.xls")

# Datos de cartografía de 2011
sacar_datos(current_dir + '/datos/cartografia_2011/', 'cartografia_censo2011_nacional.zip',
           "https://www.ine.es/censos2011_datos/cartografia_censo2011_nacional.zip")

In [12]:
os.rename(current_dir + '/datos/renta_demografía_2016/30824.csv', 
          current_dir + '/datos/renta_demografía_2016/renta_media.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30832.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_general.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30831.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_nacionalidad.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30830.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_edad.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30828.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_nacionalidad.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30827.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_edad.csv')

**IMPORTANTE:** para el archivo ``indicadores_seccen_rejilla_.xls`` hay que crear un nuevo documento, ``indicadores.csv``, con los contenidos de la segunda hoja del excels. Los contenidos quedan así.

In [4]:
indicadores = pd.read_csv(current_dir + '/datos/censos2011_datos/indicadores.csv', encoding='latin1', sep=';', header=None)
indicadores = indicadores.set_index(0)

## Procesado de datos
Ahora vamos a procesar los datos para que queden más legibles y sean más sencillos de manipular.

### Procesado de datos electorales

Para procesar los datos electorales se siguen los siguientes pasos:
- Abrirmos cada archivo ``.DAT`` y asignar nombre de columna a cada elemento de la tabla. Los elementos de las tablas vienen referenciados en el archivo ``FICHEROS.doc``.
- Procesamos los códigos de candidatura. Para procesarlos, los nombres de candidaturas similares se reducen al mismo nombre. Por ejemplo, ``UNIDOS PODEMOS``, ``UNIDAS PODEMOS`` y ``PODEMOS-IU`` se engloban dentro de ``PODEMOS``. Los resultados de candidaturas diferentes se suman.
- Unificamos los datos de los votos por candidatura, que están en el fichero 10, y los recuentos de votos, votos nulos y votos en blanco, que están en el fichero 9. 
- Reducimos el número de filas quedándonos sólo con la información de las secciones, pues es al final con la que acabaremos trabajando.
- Creamos una columna ``codigo`` que incluya la información de provincia, municipio, distrito y sección para luego cruzarla con otas tablas.

Este proceso puede tardar un rato (10 minutos con cada elección).

El archivo final es el archivo `resultados_candidaturas.pickle`, que está comprimido. Para ello usaremos el módulo `pandas`.

In [3]:
from eec import procesar_elecciones

procesar_elecciones(carpeta=current_dir + '/datos/congreso_2019_04/', sufijo='021904')
procesar_elecciones(carpeta=current_dir + '/datos/congreso_2016_06/', sufijo='021606')
procesar_elecciones(carpeta=current_dir + '/datos/congreso_2015_12/', sufijo='021512')

Procesando archivo 3
Procesando archivo 4
Procesando archivo 5
Procesando archivo 6
Procesando archivo 7
Procesando archivo 8
Procesando archivo 9
Procesando archivo 10


100%|█████████████████████████████████████████████████████████████████████████| 36370/36370 [00:01<00:00, 21145.86it/s]


Procesando archivo 3
Procesando archivo 4
Procesando archivo 5
Procesando archivo 6
Procesando archivo 7
Procesando archivo 8
Procesando archivo 9
Procesando archivo 10


100%|█████████████████████████████████████████████████████████████████████████| 36246/36246 [00:01<00:00, 20374.23it/s]


Procesando archivo 3
Procesando archivo 4
Procesando archivo 5
Procesando archivo 6
Procesando archivo 7
Procesando archivo 8
Procesando archivo 9
Procesando archivo 10


100%|█████████████████████████████████████████████████████████████████████████| 36270/36270 [00:01<00:00, 21276.53it/s]


### Procesado de rentas y calificadores demográficos
El formato del archivo es similar al de los datos electorales, indexados por el código de municipio.
Por practicidad, hemos incluido las columnas de los documentos como `pandas.MultiIndex` de modo que se puede hacer una selección de la columna como `df[('A', 'B', 'C')]`.

In [3]:
from eec import procesar_renta

procesar_renta(carpeta=current_dir + '/datos/renta_demografía_2016/', años=['2016', '2015'])

 24%|██████████████████▋                                                            | 322/1359 [00:04<00:15, 65.35it/s]D:\Programas\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
D:\Programas\anaconda3\lib\site-packages\eec\procesar_renta_demografia.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_rest['Renta media por persona'].loc[row] = np.NaN
D:\Programas\anaconda3\lib\site-packages\eec\procesar_renta_demografia.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [ ]:
from eec import procesar_demografia

procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_general.csv', 
                   ['categoria', 'año'])
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_nacionalidad.csv', 
                   ['sexo', 'nacionalidad', 'renta', 'año'])
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_nacionalidad.csv', 
                   ['sexo', 'nacionalidad', 'renta', 'año'])
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_edad.csv',
                   ['sexo', 'edad', 'renta', 'año'])
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_nacionalidad.csv',
                   ['sexo', 'nacionalidad', 'renta', 'año'])
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_edad.csv',
                   ['sexo', 'edad', 'renta', 'año'])

### Procesado de datos geográficos
Para procesar los datos geográficos vamos a necesitar pasar los datos del formato `shapefile` al formato `geojson`. Este último nos va a permitir colorear las secciones y cargarlas en mapas. Para esto, necesitaremos usar la [siguiente web](https://mapshaper.org/), donde subimos el archivo, y lo exportamos como `geojson`. Antes de exportar hacemos dos cosas:
- En la consola escribimos `-proj EPSG:4326`. Esto transforma las coordenadas del mapa a un sistema más internacional y conocido.
- En `simplify` checkeamos `prevent shape removal`, le damos a `apply` y deslizamos el deslizador hasta 50. Cuando salga a la izquierda `X line intersections`, le damos a `repair`.

Guardamos el archivo resultante `SECC_CPV_E_20111101_01_R_INE.json` en la carpeta del `shapefile`.

Como el archivo es bastante grande, para reducir el espacio vamos a reducir las coordenadas de los polígonos, y añadir campos extra.

In [208]:
# from shapely.geometry import Point, Polygon
import geopandas as gpd

censo_2011 = gpd.read_file(current_dir + '/datos/cartografia_2011/SECC_CPV_E_20111101_01_R_INE.json')

In [209]:
censo_2011['CUDIS'] = censo_2011['CUMUN'] + censo_2011['CDIS']

Vamos a crear datasets en niveles superiores, y luego vamos a guardar las columnas relevantes para cada dataset.

In [210]:
['CUSEC', 'CUMUN', 'CSEC', 'CDIS', 'CMUN', 'CUDIS', 'CCA', 'CPRO']

['CUSEC', 'CUMUN', 'CSEC', 'CDIS', 'CMUN', 'CUDIS', 'CCA', 'CPRO']

In [211]:
distritos_2011 = censo_2011.dissolve('CUDIS', as_index=False)
municipios_2011 = distritos_2011.dissolve('CUMUN', as_index=False)
provincias_2011 = municipios_2011.dissolve('CPRO', as_index=False)
autonomias_2011 = municipios_2011.dissolve('CCA', as_index=False)

In [212]:
censo_2011 = censo_2011[['OBJECTID', 'CUSEC', 'geometry']]
distritos_2011 = distritos_2011[['OBJECTID', 'CUDIS', 'geometry']]
municipios_2011 = municipios_2011[['OBJECTID', 'CUMUN', 'geometry']]
provincias_2011 = provincias_2011[['OBJECTID', 'CPRO', 'geometry']]
autonomias_2011 = autonomias_2011[['OBJECTID', 'CCA', 'geometry']]

In [213]:
for gs in [censo_2011, distritos_2011, municipios_2011, provincias_2011, autonomias_2011]:
    gs = gs.reset_index(drop=True)

In [214]:
censo_2011.to_file(current_dir + '/datos/cartografia_2011/censo_2011.json', driver='GeoJSON')
distritos_2011.to_file(current_dir + '/datos/cartografia_2011/distritos_2011.json', driver='GeoJSON')
municipios_2011.to_file(current_dir + '/datos/cartografia_2011/municipios_2011.json', driver='GeoJSON')
provincias_2011.to_file(current_dir + '/datos/cartografia_2011/provincias_2011.json', driver='GeoJSON')
autonomias_2011.to_file(current_dir + '/datos/cartografia_2011/autonomias_2011.json', driver='GeoJSON')

ERROR:fiona._env:C:\Users\alexm\Dropbox\Otros\espana_en_cifras/datos/cartografia_2011/autonomias_2011.json: No such file or directory


Opcionalmente, y sobre todo para los mapas más grandes, podemos reducir el tamaño de los archivos como lo hemos hecho inicialmente para el archivo de censos.